In [1]:
%pip install cryptography
!pip uninstall pycrypto -y
!pip3 install pycryptodome pmlb

Note: you may need to restart the kernel to use updated packages.


In [2]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.serialization import Encoding, ParameterFormat, PublicFormat
from cryptography.hazmat.primitives.serialization import load_pem_parameters, load_pem_public_key
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

### Key exchange

In [3]:
parameters = dh.generate_parameters(generator=2, key_size=1024, backend=default_backend())

with open("shared_file", "wb") as file:
    file.write(parameters.parameter_bytes(encoding=Encoding.PEM, format=ParameterFormat.PKCS3))

In [4]:
with open ("shared_file", "rb") as file:
    public_mixed_with_D = load_pem_public_key(file.read()) 

private_key_C = parameters.generate_private_key() # red
public_mixed_with_C  = private_key_C.public_key()  #orange

with open("shared_file", "wb") as file:
    file.write(public_mixed_with_C.public_bytes(encoding=Encoding.PEM, format=PublicFormat.SubjectPublicKeyInfo))

In [5]:
shared_key_D_then_C = private_key_C.exchange(public_mixed_with_D) # brown

derived_key_D_then_C = HKDF(
    algorithm = hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
    backend=default_backend()
).derive(shared_key_D_then_C)

print(derived_key_D_then_C)

b'\xaf8Z5\xf23\xfa\x95\xdb\x9c\xba\xf4\x81\x95)\xf7\xf3\x850\xcc\xa4\xaal\xa9\xdb\x07y\xd5\xb9~\xcd\x13'


### Encryption without Integrity and Authenticity 

In [32]:
from Crypto.Cipher import AES
from Crypto.Util import Counter

In [33]:
def encrypt_csv_ctr(file_name, civ, counter_size=128):
    
    counter = Counter.new(counter_size, initial_value=civ)
    
    aes = AES.new(derived_key_D_then_C, mode=AES.MODE_CTR, counter=counter)

    with open(file_name, 'rb') as file:
        csv_data = file.read()
        encrypted_csv = aes.encrypt(csv_data)
    return encrypted_csv

columns_desc_enc = encrypt_csv_ctr("data/columns_description-CORRECTED.csv", 10)

with open("shared_file", "wb") as file:
    file.write(columns_desc_enc)

In [ ]:
infringement_dataset_enc = encrypt_csv_ctr("data/infringement_dataset.csv", 10)

with open("shared_file", "wb") as file:
    file.write(infringement_dataset_enc)

### Encryption with Integrity and Authenticity 

In [38]:
import os
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

In [39]:
with open("data/columns_description-CORRECTED.csv", 'rb') as file:
        columns_desc = file.read()

aesgcm = AESGCM(derived_key_D_then_C)

nonce = os.urandom(12)
columns_desc_enc = aesgcm.encrypt(nonce, columns_desc, None) # your code here

with open("shared_file", "wb") as file:
    file.write(columns_desc_enc)

In [40]:
with open("shared_file", "wb") as file:
    file.write(nonce)

In [ ]:
with open("data/infringement_dataset.csv", 'rb') as file:
        infringement_dataset = file.read()

nonce = os.urandom(12)
infringement_dataset_enc = aesgcm.encrypt(nonce, infringement_dataset, None) # your code here

with open("shared_file", "wb") as file:
    file.write(infringement_dataset_enc)

In [ ]:
with open("shared_file", "wb") as file:
    file.write(nonce)